# Brick

In [1]:
from llmbrick.core.brick import BaseBrick
import nest_asyncio

class LLMBrick(BaseBrick[str, str]):
    pass


llm = LLMBrick()

@llm.unary()
async def input(prompt: str) -> str:
    """
    Input a string from the user.

    Args:
        prompt (str): The prompt to display to the user.

    Returns:
        str: The input string provided by the user.
    """
    return f"user input: {prompt}"


    # 在 Jupyter Notebook 中運行 async 函數可以使用 asyncio.run 或 nest_asyncio
nest_asyncio.apply()

result = await llm.run_unary("What is your name? ")
print(result)


2025-07-24 20:48:11 | INFO    2301058 | llmbrick:async_wrapper:108 - [LLMBrick-Unary] [wrapper] input: args=('What is your name? ',), kwargs={}
2025-07-24 20:48:11 | INFO    2301058 | llmbrick:async_wrapper:112 - [LLMBrick-Unary] [wrapper] output: user input: What is your name? 


Logger 'llmbrick' (ID: llmbrick): Console only mode
user input: What is your name? 


In [ ]:
result = await llm.run_unary("請輸入內容：")
print(result)

In [2]:
from llmbrick.core.brick import BaseBrick, unary_handler
import nest_asyncio


class MyNewBrick(BaseBrick[str, str]):
    def __init__(self, some_param: str, **kwargs):
        super().__init__(**kwargs)
        self.some_param = some_param
    
    @unary_handler
    async def process(self, input_data: str) -> str:
        """
        Process the input data and return a string.

        Args:
            input_data (str): The input data to process.

        Returns:
            str: The processed output.
        """
        return f"Processed: {input_data} with param {self.some_param}"
    
nest_asyncio.apply()

brick = MyNewBrick(some_param="example")

result = await brick.run_unary("What is your name? ")
print(result)


Processed: What is your name?  with param example


In [3]:
from llmbrick.bricks.llm.base_llm import LLMBrick

my_llm = LLMBrick("123123")
@my_llm.bidi_streaming()
async def bidi_streaming_handler(input_stream):
    async for input_data in input_stream:
        yield f"Processed: {input_data}"


/tmp/ipykernel_2301058/2243463879.py:4: DeprecationWarning: Call to deprecated method bidi_streaming. (LLMBrick does not support bidi_streaming handler.)
  @my_llm.bidi_streaming()


NotImplementedError: LLMBrick does not support bidi_streaming handler.

# Logging

In [4]:
from llmbrick.utils.logging import log_function

# info: This will log the function call with the specified service name and level.
@log_function(service_name="math_service", level="info")
def add(a: int, b: int) -> int:
    return a + b

r1 = add(1, 2)

# warning: This will log the function call with the specified service name and level.
@log_function(service_name="math_service", level="warning")
def subtract(a: int, b: int) -> int:
    return a - b

r2 = subtract(5, 3)


# error: This will log the function
@log_function(service_name="math_service", level="error")
def divide(a: int, b: int) -> float:
    if b == 0:
        raise ValueError("Division by zero is not allowed.")
    return a / b

try:
    r2 = divide(10, 0)
except ValueError as e:
    print(f"Error occurred: {e}")  

2025-07-24 20:48:26 | INFO    2301058 | llmbrick:sync_wrapper:122 - [math_service] [add] input: args=(1, 2), kwargs={}
2025-07-24 20:48:26 | INFO    2301058 | llmbrick:sync_wrapper:126 - [math_service] [add] output: 3
2025-07-24 20:48:26 | WARNING 2301058 | llmbrick:sync_wrapper:122 - [math_service] [subtract] input: args=(5, 3), kwargs={}
2025-07-24 20:48:26 | WARNING 2301058 | llmbrick:sync_wrapper:126 - [math_service] [subtract] output: 2
2025-07-24 20:48:26 | ERROR   2301058 | llmbrick:sync_wrapper:122 - [math_service] [divide] input: args=(10, 0), kwargs={}
2025-07-24 20:48:26 | ERROR   2301058 | llmbrick:sync_wrapper:130 - [math_service] [divide] exception: Division by zero is not allowed.


Error occurred: Division by zero is not allowed.
